In [10]:
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt

a = np.array([1,2,3,4,5,6,7,8])
v = np.array([1,1,1,1])

print(np.convolve(a,v,'valid')/4)

np.random.multivariate_normal(np.array([5]),cov=np.array([1]))




[2.5 3.5 4.5 5.5 6.5]


ValueError: cov must be 2 dimensional and square